In [ ]:
###########################################################
#                                                         #
#               TRAITEMENT SIMUS FORTRAN                  #
#                     STATISTIQUES                        #
#                                                         #
#                         avec                            #
#                   - lecture arborescence                #
#                  analyse                                #
#                   - Delta Fluo                          #
#                   - Delta Energy GMol                   #
#                   - R/L Ca+                             #
#                   - distance inter ionique              #
#                   - trajectoire GMol                    #
#                                                         #
#                          02/2020     Adrien Poindron    #
###########################################################

In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
matplotlib.rcParams['figure.figsize'] = [10.5,7]
matplotlib.rcParams.update({'font.size': 25})

# Imports

In [4]:
# %pylab

import numpy
import matplotlib
from matplotlib.patches import Ellipse
from matplotlib import pylab, mlab, pyplot
from mpl_toolkits.mplot3d import Axes3D
np = numpy
plt = pyplot

matplotlib.rcParams.update({'font.size': 21})
# matplotlib.rcParams["figure.dpi"] = 300

from IPython.display import display
from IPython.core.pylabtools import figsize, getfigs

from pylab import *
from numpy import *

from scipy.optimize import curve_fit

In [5]:
import os
from os import listdir
from os.path import isfile, join

import tkinter as tk
from tkinter import filedialog
%matplotlib qt

import matplotlib.patches as patches

from pathlib import Path
from matplotlib import gridspec
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes
from mpl_toolkits.axes_grid1.inset_locator import mark_inset

In [6]:
from natsort import natsorted   # natural sort marche de ouf
import re                       # find '/' in strings
import timeit
cm = pylab.get_cmap('summer')   # colormap par défaut color=cm(k/len(array))

In [7]:
# Fonctions persos de Jofre et Adrien

# %run function_jofre.ipy
%run "../../Functions/1.1/data_fct_Adrien_point_by_point.py"

In [8]:
# Constantes de la physique
# ABSOLUMENT RECHARGER APRÈS AVOIR EXECUTÉ LES CASES D'IMPORT AU DESSUS

C_e = 1.602e-19        # Coulomb
kb = 1.38064852*1e-23  # Boltzman
m_Ca = 40.078*1.66054e-27 # masse Ca 40.078
m_GM = 1e6*1.66054e-27 # la masse de la GMol
eps0 = 8.854187*1e-12  # permittivité électrique du vide

r0 = 2.5e-3 # 2.5e-3   # rayon piège Ca+
d0 = 4e-3/2            # longueur piège Ca+

Omega = 2.05e6*2*pi # 2.047e6

bk = 4 # nombre de barreaux par groupe (2 -> 4-pole , 4 -> 8-pole ...)

mkappa = 0.23          # écrantage piège réel GiantMol
wzLC = (2*pi*90806.9982303)**2
kappa_simion = m_Ca*d0**2*wzLC/(2*C_e)
print('%s = %f' % ('$\kappa_{simion}$',kappa_simion) )

zeta = kappa_simion*r0**2/d0**2

$\kappa_{simion}$ = 0.270471


# Sélection fichier

In [9]:
## GUI for data loading
# Select one data file all the way down to the directories
# Temp_SimuType0_N01024_Vrf0064_Udc0.5000D+00V_D1.0_S1.0RFG.dat

file_cfg, slash_cfg, all_subdir = load_gui(filter_nocomplete=1)

file_path = file_cfg[0]
dir_path = file_cfg[1]
work_rep = file_cfg[2]
filename = file_cfg[3]

myslashpos = slash_cfg[0]
slashcond = slash_cfg[1]

> Répertoire : /home/adrian/RemoteFS/Hobitton/20200909_1/DC01_RF10/Try00
> Filename : Temp_SimuType0_N01024_Vrf0064_Udc0.1000D+01V_D2.0_S1.0RFG.dat
> myslashpos | [0, 5, 12, 21, 30, 41, 51]
> slashcond | 5
Points deleted because they were not complete []   0 pt(s)
Total number of data directories 2


In [10]:
## Condition for data
points_and_coord, condition_parameters = simu_conditions(all_subdir, myslashpos, slashcond, filename)

> condition names ['DC', 'RF']
> number of points 2
000 > DC01_RF08
001 > DC01_RF10
> N_ions = 1024
> e_GMol = 50


In [11]:
## Data loading (1D arrays)
data_name, num_runs, PMandT, Gmol_data, r_LC_clip, dim_nu \
= data_retrieve(all_subdir, points_and_coord, condition_parameters, slash_cfg) 

data0 = data_name[0]
data2 = data_name[1]
data4 = data_name[2]

PMvar = PMandT[0]
Tvar = PMandT[1]

deltaEc = Gmol_data[0]
deltaEcRel = Gmol_data[1]
t_c = Gmol_data[2]

> Points | 2
> Simulations pour chaque point | ['Try00']
Hello
Point n° 0
00 - 00  >  /home/adrian/RemoteFS/Hobitton/20200909_1/DC01_RF08/Try00/Temp_SimuType0_N01024_Vrf0053_Udc0.1000D+01V_D2.0_S1.0RFG
Point n° 1
01 - 00  >  /home/adrian/RemoteFS/Hobitton/20200909_1/DC01_RF10/Try00/Temp_SimuType0_N01024_Vrf0064_Udc0.1000D+01V_D2.0_S1.0RFG
Time elapsed:  1.3910229999999997 s
Time elapsed:  0.023183716666666663 m


# Analyser un seul point et tous ses essais

In [12]:
# Récupérer les données pour un point

# tt                  : temps
# PM, SNR             : signal PM temporel, rapport signal/bruit, ci.e fluo/bruitfluo
# PMvar, Tvar         : variation de fluo et température entre passage Gmol et fin de simu
# deltaEc, deltaEcRel : variation énergie GMol
# T_aux, T_CM         : température interne du nuage, température du centre de masse
# r_LC, v_LC, a_LC    : position, vitesse, accélération des ions Ca+
# r_LC_clip           : même chose mais en dégageant les ions perdus et beaucoup trop loin (hors piège)
# dim_nu              : dimensions x, y et z, nuage Ca+ basé sur r_LC_clip

myslashpos = slash_cfg[0]
slashcond = slash_cfg[1]

# determining number of elements on each repetition
num_runs = [runs[myslashpos[slashcond+1]+1:] for runs in all_subdir if list(points_and_coord.keys())[0] in runs]
num_runs = list(dict.fromkeys(num_runs))

cond_name = 'DC01_RF10'
pt_to_plot = [i for i in all_subdir if cond_name in i]
for k, address in enumerate(pt_to_plot):
    onlyfiles = [f for f in listdir(address) if isfile(join(address, f)) and not "xva" in f and ".dat" in f]
fluo_var, GMol_var, cloud_atlas, t_aux, r_LC_clip = load_T_PM_cloud_GMol(all_subdir[0],flag_plot=1)
        
shapevar = (len(pt_to_plot),len(cloud_atlas[2]))
PMvar = np.zeros(shapevar)
Tvar = np.zeros(shapevar)
SNR = np.zeros(shapevar)
tt = np.zeros(shapevar)
T_aux = np.zeros((len(pt_to_plot),len(cloud_atlas[2]),3))
T_CM = np.zeros((len(pt_to_plot),len(cloud_atlas[2]),3))
PM = np.zeros(shapevar)
        
t0 = time.clock()
print("Hello")
    
for k, address in enumerate(pt_to_plot):
    fluo_var, GMol_var, cloud_atlas, t_aux, r_LC_clip = load_T_PM_cloud_GMol(address,flag_plot=0)
    PMvar[k] = fluo_var[0] ; Tvar[k] = fluo_var[1] ; SNR[k] = fluo_var[2]
    tt[k] = cloud_atlas[0] ; T_aux[k] = cloud_atlas[1] ; T_CM[k] = cloud_atlas[2] ; PM[k] = cloud_atlas[3]

t1 = time.clock() - t0
print("Time elapsed: ", t1, 's') # CPU seconds elapsed (floating point)
print("Time elapsed: ", t1/60, 'm') # CPU seconds elapsed (floating point)

Hello
Time elapsed:  0.34874000000000027 s
Time elapsed:  0.005812333333333338 m


In [ ]:
# Plot all temperatures rf relax après refroidissement

fig, ax = plt.subplots(4, 4,num='Temperature interaction Gmol Ca+',clear=True,sharex=True,sharey=True)
fig.set_size_inches(11.69,8.27)
# suptitle('Saturation 866')
to_plot = 0

for l in range(0,4):
    for k in range(0,4):
        ax[k,l].plot(tt[to_plot,:]*1e3,T_aux[to_plot,:,0],'+',mew=1,ms=5)
        ax[k,l].plot(tt[to_plot,:]*1e3,T_aux[to_plot,:,1],'+',mew=1,ms=5)
        ax[k,l].plot(tt[to_plot,:]*1e3,T_aux[to_plot,:,2],'+',mew=1,ms=5)
        ax[k,l].vlines(t_aux[0]*1e3,1e-4,1e-1,color='red')
        ax[k,l].vlines(t_aux[1]*1e3,1e-4,1e-1,color='red')
        ax[k,l].set_yticks([1e-4,1e-3,1e-2,1e-1,1,1e2,1e4])
        ax[k,l].grid()
    #     ax[k].set_ylim(-0.1,0.55)
        ax[k,l].xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=2))
        ax[k,l].annotate('try'+str(to_plot), xy=(0.80, 0.99), xycoords='axes fraction', # pts_concerned[to_plot]
                size=12, ha='left', va='top',
                bbox=dict(boxstyle='round', fc='w'))
        to_plot += 1
        if to_plot == 13:
            break
    if to_plot == 13:
        break
# ax[3].set_xlim(-10,4000)
# ax[k,l].set_ylim(2e-4,1e-1)
fig.text(0.56, 0.04, 'time [ms]', ha='center')
fig.text(0.04, 0.54, 'T [K]', va='center', rotation='vertical')

fig.suptitle(pt_to_plot[0][:-6],fontsize=10)

# fig.text(0.5, 0.01, 'f', ha='center')
# fig.text(0.01, 0.5, r'$| FFT(HC(t)) | / N_{sample}$', va='center', rotation='vertical')

fig.set_size_inches(11.69,8.27)
plt.tight_layout()
subplots_adjust(hspace=0.05, wspace = 0.05, bottom = 0.120, left = 0.140) # hspace=0, top=0.95, left = 0.065, bottom = 0.07
# savefig('FFT_lock_oscillo_0_11500.png')

In [ ]:
fig, ax = plt.subplots(1, 1,num='one temp',clear=True,sharex=True,sharey=True)
fig.set_size_inches(11.69,8.27)
# suptitle('Saturation 866')

plot(tt[to_plot,:]*1e3,PM[to_plot,:],'+',mew=1,ms=5)
plot(tt[to_plot,:]*1e3,PM[to_plot,:],'+',mew=1,ms=5)
plot(tt[to_plot,:]*1e3,PM[to_plot,:],'+',mew=1,ms=5)
grid()

In [ ]:
# Plot all fluo rf relax après refroidissement

fig, ax = plt.subplots(4, 4,num='Fluo interaction Gmol Ca+',clear=True,sharex=True,sharey=True)
fig.set_size_inches(11.69,8.27)
# suptitle('Saturation 866')
to_plot = 0

for l in range(0,4):
    for k in range(0,4):
        ax[k,l].plot(tt[to_plot,:]*1e3,PM[to_plot,:],'+',mew=1,ms=5)
        ax[k,l].plot(tt[to_plot,:]*1e3,PM[to_plot,:],'+',mew=1,ms=5)
        ax[k,l].plot(tt[to_plot,:]*1e3,PM[to_plot,:],'+',mew=1,ms=5)
        ax[k,l].vlines(t_aux[0]*1e3,1e-4,1e-1,color='red')
        ax[k,l].vlines(t_aux[1]*1e3,1e-4,1e-1,color='red')
        ax[k,l].set_yticks([1e-4,1e-3,1e-2,1e-1,1,1e2,1e4])
        ax[k,l].grid()
    #     ax[k].set_ylim(-0.1,0.55)
        ax[k,l].xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=2))
        ax[k,l].annotate('try'+str(to_plot), xy=(0.80, 0.99), xycoords='axes fraction', # pts_concerned[to_plot]
                size=12, ha='left', va='top',
                bbox=dict(boxstyle='round', fc='w'))
        to_plot += 1
        if to_plot == 13:
            break
    if to_plot == 13:
        break
# ax[3].set_xlim(-10,4000)
# ax[k,l].set_ylim(2e-4,1e-1)
fig.text(0.56, 0.04, 'time [ms]', ha='center')
fig.text(0.04, 0.54, 'T [K]', va='center', rotation='vertical')

fig.suptitle(pt_to_plot[0][:-6],fontsize=10)

# fig.text(0.5, 0.01, 'f', ha='center')
# fig.text(0.01, 0.5, r'$| FFT(HC(t)) | / N_{sample}$', va='center', rotation='vertical')

fig.set_size_inches(11.69,8.27)
plt.tight_layout()
subplots_adjust(hspace=0.05, wspace = 0.05, bottom = 0.120, left = 0.140) # hspace=0, top=0.95, left = 0.065, bottom = 0.07
# savefig('FFT_lock_oscillo_0_11500.png')

In [ ]:
# position des ions à la fin de la simu
to_plot = 9
onlyfiles = [f for f in listdir(pt_to_plot[to_plot]) if isfile(join(pt_to_plot[to_plot], f)) and "xva" in f and ".bin" in f]
print(sort(onlyfiles))
file_name = '{}/xva_{}'.format(pt_to_plot[to_plot],sort(onlyfiles)[0].strip('.bin')[4:])
plot_XYZ(file_name,fig_name='XYZ_point'+str(pt_to_plot[to_plot]),fig_title='XYZ')
print(sort(onlyfiles)[2])

In [ ]:
# pos ions par rapport à potentiel axial

Udc = 2
wz_LC = 2*np.pi*90806.9982303
wz2   = Udc*wz_LC**2 # * C_e*m_Ca / (C_e*m_GM)
eta = 1. / (4.* (sqrt(10) + 5. ) )
L = 0.0140608827209
cte_gauss = -1/(2*eta*L**2)

r_LC,v_LC,a_LC = load_xyz_init_bin_DP(file_name)

r_z = linspace(-0.02,0.02,1000)
gauss_pot = wz2*eta*L**2*m_Ca * (1-exp(cte_gauss*r_z**2)) /C_e
quad_pot = 0.5*wz2*m_Ca*r_z**2 /C_e

fig = figure('axial_potential');clf()
ax0 = fig.add_subplot(111)
title(pt_to_plot[0][:-6])

ax0.plot(r_z*1e3,gauss_pot,color='C0')
ax0.plot(r_z*1e3,quad_pot,color='C1')
ax0.plot(r_LC[2],r_LC[0]-min(r_LC[0]),ls='',marker='o',color='C4')

ax0.set_xlabel('z [mm]')
ax0.set_ylabel('$E_p$ [eV]')

ax0.grid()

tight_layout()

In [ ]:
z0 = 400*1e-6
print( (-3/(eta*L**2)/12)*z0**2 * 0.5*m_Ca*wz2*z0**2 / C_e )

# Analyser tous les cas de SimuType6

In [ ]:
# Récupérer les données pour les SimuType6

# tt                  : temps
# PM, SNR             : signal PM temporel, rapport signal/bruit, ci.e fluo/bruitfluo
# PMvar, Tvar         : variation de fluo et température entre passage Gmol et fin de simu
# deltaEc, deltaEcRel : variation énergie GMol
# T_aux, T_CM         : température interne du nuage, température du centre de masse
# r_LC, v_LC, a_LC    : position, vitesse, accélération des ions Ca+
# r_LC_clip           : même chose mais en dégageant les ions perdus et beaucoup trop loin (hors piège)
# dim_nu              : dimensions x, y et z, nuage Ca+ basé sur r_LC_clip

myslashpos = slash_cfg[0]
slashcond = slash_cfg[1]

# determining number of elements on each repetition
num_runs = [runs[myslashpos[slashcond+1]+1:] for runs in all_subdir if list(points_and_coord.keys())[0] in runs]
num_runs = list(dict.fromkeys(num_runs))

# Variables à deux coordonnées : [point, try]

simutype6address = []
pts_concerned = []
for k, address in enumerate(all_subdir):
    onlyfiles = [f for f in listdir(address) if isfile(join(address, f)) and not "xva" in f and ".dat" in f]
    onlySimuType6 = [i for i in onlyfiles if 'SimuType6' in i]
    print(k,'- ', onlySimuType6)
    if onlySimuType6 != []:
        simutype6address.append(address)
        pts_concerned.append( re.sub('/','',all_subdir[k][myslashpos[slashcond]:myslashpos[slashcond+1]]) )
fluo_var, GMol_var, cloud_atlas, t_aux, r_LC_clip = load_RF_relax(all_subdir[0],flag_plot=0,RF_relax_type='AfterCool')
        
shapevar = (len(simutype6address),len(cloud_atlas[2]))
PMvar = np.zeros(shapevar)
Tvar = np.zeros(shapevar)
SNR = np.zeros(shapevar)
tt = np.zeros(shapevar)
T_aux = np.zeros((len(simutype6address),len(cloud_atlas[2]),3))
T_CM = np.zeros((len(simutype6address),len(cloud_atlas[2]),3))
PM = np.zeros(shapevar)
        
t0 = time.clock()
print("Hello")
    
for k, address in enumerate(simutype6address):
    fluo_var, GMol_var, cloud_atlas, t_aux, r_LC_clip = load_RF_relax(address,flag_plot=0,RF_relax_type='AfterCool')
    PMvar[k] = fluo_var[0] ; Tvar[k] = fluo_var[1] ; SNR[k] = fluo_var[2]
    tt[k] = cloud_atlas[0] ; T_aux[k] = cloud_atlas[1] ; T_CM[k] = cloud_atlas[2] ; PM[k] = cloud_atlas[3]

t1 = time.clock() - t0
print("Time elapsed: ", t1, 's') # CPU seconds elapsed (floating point)
print("Time elapsed: ", t1/60, 'm') # CPU seconds elapsed (floating point)

In [ ]:
# position des ions à la fin de la simu
to_plot = 13
onlyfiles = [f for f in listdir(simutype6address[to_plot]) if isfile(join(simutype6address[to_plot], f)) and "xva" in f and ".bin" in f]
print(sort(onlyfiles))
file_name = '{}/xva_{}'.format(simutype6address[to_plot],sort(onlyfiles)[3].strip('.bin')[4:])
plot_XYZ(file_name,fig_name='XYZ_point'+str(simutype6address[to_plot]),fig_title='XYZ')
print(sort(onlyfiles)[2])

In [ ]:
# Plot all temperatures rf relax après refroidissement

fig, ax = plt.subplots(4, 7,num='Temperature RF relax sans injection',clear=True,sharex=True,sharey=True)
fig.set_size_inches(11.69,8.27)
# suptitle('Saturation 866')
to_plot = 0
for l in range(0,7):
    for k in range(0,4):
        ax[3-k,l].semilogy(tt[to_plot,:]*1e3,T_aux[to_plot,:,0],'+',mew=1,ms=5)
        ax[3-k,l].semilogy(tt[to_plot,:]*1e3,T_aux[to_plot,:,1],'+',mew=1,ms=5)
        ax[3-k,l].semilogy(tt[to_plot,:]*1e3,T_aux[to_plot,:,2],'+',mew=1,ms=5)
        ax[3-k,l].vlines(t_aux[0]*1e3,1e-4,1e-1,color='red')
        ax[3-k,l].vlines(t_aux[1]*1e3,1e-4,1e-1,color='red')
        ax[3-k,l].set_yticks([1e-4,1e-3,1e-2,1e-1,1,1e2,1e4])
        ax[3-k,l].grid()
    #     ax[k].set_ylim(-0.1,0.55)
        ax[3-k,l].xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=2))
        ax[3-k,l].annotate(pts_concerned[to_plot], xy=(0.29, 0.99), xycoords='axes fraction',
                size=12, ha='left', va='top',
                bbox=dict(boxstyle='round', fc='w'))
        to_plot += 1
# ax[3].set_xlim(-10,4000)
ax[3-k,l].set_ylim(2e-4,1e-1)
fig.text(0.56, 0.04, 'time [ms]', ha='center')
fig.text(0.04, 0.54, 'T [K]', va='center', rotation='vertical')

fig.suptitle(all_subdir[0],fontsize=10)

# fig.text(0.5, 0.01, 'f', ha='center')
# fig.text(0.01, 0.5, r'$| FFT(HC(t)) | / N_{sample}$', va='center', rotation='vertical')

plt.tight_layout()
subplots_adjust(hspace=0.05, wspace = 0.05, bottom = 0.120, left = 0.140) # hspace=0, top=0.95, left = 0.065, bottom = 0.07
# savefig('FFT_lock_oscillo_0_11500.png')
# fig.set_size_inches(11.69,8.27)

In [ ]:
# Plot all temperatures Centres de Masse rf relax après refroidissement

fig, ax = plt.subplots(4, 7,num='Temperature CM RF relax sans injection',clear=True,sharex=True,sharey=True)
fig.set_size_inches(11.69,8.27)
# suptitle('Saturation 866')
to_plot = 0
for l in range(0,7):
    for k in range(0,4):
        ax[3-k,l].semilogy(tt[to_plot,:]*1e3,T_CM[to_plot,:,0],'+',mew=1,ms=5)
        ax[3-k,l].semilogy(tt[to_plot,:]*1e3,T_CM[to_plot,:,1],'+',mew=1,ms=5)
        ax[3-k,l].semilogy(tt[to_plot,:]*1e3,T_CM[to_plot,:,2],'+',mew=1,ms=5)
        ax[3-k,l].semilogy(tt[to_plot,:]*1e3,T_aux[to_plot,:,0],'.',mew=1,ms=5,color='y')
        ax[3-k,l].vlines(t_aux[0]*1e3,1e-4,1e-1,color='red')
        ax[3-k,l].vlines(t_aux[1]*1e3,1e-4,1e-1,color='red')
#         ax[3-k,l].set_yticks([1e-4,1e-3,1e-2,1e-1,1,1e2,1e4])
        ax[3-k,l].grid()
    #     ax[k].set_ylim(-0.1,0.55)
        ax[3-k,l].xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=2))
        ax[3-k,l].annotate(pts_concerned[to_plot], xy=(0.29, 0.99), xycoords='axes fraction',
                size=12, ha='left', va='top',
                bbox=dict(boxstyle='round', fc='w'))
        to_plot += 1
# ax[3].set_xlim(-10,4000)
ax[3-k,l].set_ylim(1e-12,1e3)
fig.text(0.56, 0.04, 'time [ms]', ha='center')
fig.text(0.04, 0.54, 'T [K]', va='center', rotation='vertical')

fig.suptitle(all_subdir[0],fontsize=10)

# fig.text(0.5, 0.01, 'f', ha='center')
# fig.text(0.01, 0.5, r'$| FFT(HC(t)) | / N_{sample}$', va='center', rotation='vertical')

plt.tight_layout()
subplots_adjust(hspace=0.05, wspace = 0.05, bottom = 0.120, left = 0.140) # hspace=0, top=0.95, left = 0.065, bottom = 0.07
# savefig('FFT_lock_oscillo_0_11500.png')
# fig.set_size_inches(11.69,8.27)

In [ ]:
to_plot = 25
fluo_var_one, GMol_var_one, cloud_atlas_one, t_aux_one, r_LC_clip_one = load_RF_relax(simutype6address[to_plot],flag_plot=1,RF_relax_type='AfterCool')
T_aux = cloud_atlas_one[1]
T_aux = T_aux[len(freq_one)//3:,1]
time_one = cloud_atlas_one[0]
dtime_one = time_one[50]-time_one[49]
freq_one = np.fft.fftfreq(len(T_aux),d=dtime_one)
print(dtime_one,freq_one[51]-freq_one[50])

fig = figure('FFT'); clf()
title(simutype6address[to_plot],fontsize=12)
ax1 = subplot(111)
ax1.scatter(freq_one*1e-3,abs(real(fft(T_aux*hanning(len(T_aux))))),marker='+')
# ax1.set_xlim(-10,1/1.9/dtime_one/1e3)
ax1.set_xlim(600,1020)
ax1.set_ylim(-2,12)
ax1.grid()

legend()
xlabel('f [kHz]')

fig.set_size_inches(11.69, 8.27)
plt.tight_layout()

In [ ]:
# FFT test
time_one = np.linspace(0,1,1000)
T_aux = np.sin(2*pi*25*time_one)
fig = figure('FFT test'); clf()
ax1 = subplot(211)
ax1.plot(time_one,T_aux)

dtime_one = time_one[50]-time_one[49]
freq_one = np.fft.fftfreq(len(T_aux),d=dtime_one)

ax2 = subplot(212)
ax2.scatter(freq_one,real(fft(T_aux*hanning(len(T_aux)))),marker='+')

fig.set_size_inches(11.69, 8.27)
plt.tight_layout()

In [ ]:
for k, l in enumerate(simutype6address):
    print('>',k,'-',l)

In [13]:
address = all_subdir[1]
print(address)
print(onlyfiles)

/home/adrian/RemoteFS/Hobitton/20200909_1/DC01_RF10/Try00
['Temp_SimuType4_01_N01024_Vrf0064_Udc0.1000D+01V_D2.0_S1.0RFG50eV.dat', 'Temp_SimuType2_01_N01024_Vrf0064_Udc0.1000D+01V_D2.0_S1.0RFG50eV.dat', 'Temp_SimuType0_N01024_Vrf0064_Udc0.1000D+01V_D2.0_S1.0RFG.dat']


In [ ]:
v_LC_rf[0]

In [30]:
# Recover Ca pos, speed and acc at the end of one specific step
# address = simutype6address[20]
print(address)
onlyfiles = [f for f in listdir(address) if isfile(join(address, f)) and "xva" in f and 'SimuType0' in f and 'bin' in f]
r_LC, v_LC, a_LC, v_rf_avg = load_xyz_init_bin_DP(address+'/'+onlyfiles[0].strip('.bin'))


v_LC_avg_x = v_rf_avg[0]/1000/100
v_LC_avg_y = v_rf_avg[1]/1000/100
v_LC_avg_z = v_rf_avg[2]/1000/100

my_base = 10

fig = figure('Terminal speed'); clf()
fig.suptitle(r'Atomic speed at the end of SimuType0 - no RF - '+str(address[31:]),color='xkcd:black',fontsize = 18)
ax1 = subplot(311)
my_bins = arange(-35,36,2)
ax1.hist(v_LC_avg_x,bins=my_bins ,color='C0', edgecolor='black', linewidth=1.2)
ax1.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=my_base))
ax1.grid()
xlabel('$v_x$ [m/s]')

ax2 = subplot(312)
ax2.hist(v_LC_avg_y,bins=my_bins ,color='C1', edgecolor='black', linewidth=1.2)
ax2.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=my_base))
ax2.grid()
xlabel('$v_y$ [m/s]')

my_base = 0.2
ax3 = subplot(313)
my_bins = arange(-1,1,0.1)
# my_bins = arange(-30000,30001,5000)
ax3.hist(v_LC_avg_z,bins=my_bins ,color='C2', edgecolor='black', linewidth=1.2)
ax3.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=my_base))
ax3.grid()
xlabel('$v_z$ [m/s]')

fig.set_size_inches(11.69, 8.27)
plt.tight_layout()
subplots_adjust(hspace=0.50, top = 0.93)

/home/adrian/RemoteFS/Hobitton/20200909_1/DC01_RF10/Try00


In [ ]:
# Recover Ca pos, speed and acc at the end of one specific step
# address = simutype6address[20]
print(address)
onlyfiles = [f for f in listdir(address) if isfile(join(address, f)) and "xva" in f and 'SimuType0' in f and 'bin' in f]
r_LC, v_LC, a_LC, v_LC_avg = load_xyz_init_bin_DP(address+'/'+onlyfiles[0].strip('.bin'))

my_base = 100000

fig = figure('Average Terminal speed'); clf()
fig.suptitle(r'Average atomic speed at the end of SimuType0 - '+str(address[31:]),color='xkcd:black',fontsize = 18)
ax1 = subplot(311)
my_bins = arange(-500000,500001,20000)
ax1.hist(v_LC_avg[0],bins=my_bins ,color='C0', edgecolor='black', linewidth=1.2)
ax1.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=my_base))
ax1.grid()
xlabel('$v_x$ [m/s]')

ax2 = subplot(312)
ax2.hist(v_LC_avg[1],bins=my_bins ,color='C1', edgecolor='black', linewidth=1.2)
ax2.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=my_base))
ax2.grid()
xlabel('$v_y$ [m/s]')

ax3 = subplot(313)
my_bins = arange(-25000,25001,1000)
my_base = 10000
ax3.hist(v_LC_avg[2],bins=my_bins ,color='C2', edgecolor='black', linewidth=1.2)
ax3.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=my_base))
ax3.grid()
xlabel('$v_z$ [m/s]')

fig.set_size_inches(11.69, 8.27)
plt.tight_layout()
subplots_adjust(hspace=0.50, top = 0.93)

In [ ]:
Vrf = 59.23*2
r_ion = 100*1e-6
# calculate equivalent speed in RF field for a given radius
v_ion = C_e/m_Ca/Omega*Vrf/r0**2*r_ion
print(v_ion)

v_ion = 100
# calculate equivalent radius in RF field for a given atomic speed
r_ion = m_Ca*Omega/C_e*r0**2/Vrf*v_ion
print(r_ion*1e6)